In [1]:
import json
import numpy as np
from tqdm import tqdm
from ray.rllib.algorithms.ppo import PPOConfig
from ray.tune.registry import register_env
from fury_sim_env import FurySimEnv
from gymnasium import make
from gymnasium.envs.registration import register
from gymnasium.wrappers import FrameStack
from fury import *

spell_id_lists = {
        '0' : "Melee",
        '23881' : "Bloodthirst",
        '1680' : "Whirlwind",
        '47475' : "Slam",
        '47450' : "Heroic Strike",
        '47471' : "Execute",
        '12867': "Deep Wounds",
        '12292' : "Death Wish",
        '1719' : "Recklessness",
        '64382' : "Shattering Throw",
        '54758' : "Engi Gloves",
        '2457' : "Battle Stance",
        '2458' : "Berserker Stance",
        '2687' : 'Bloodrage',
        '2825' : 'Bloodlust',
        '47465': 'Rend',
        '7384': 'Overpower',
        '44949': 'Whirlwind OH',
}

c:\ProgramData\Anaconda3\envs\civ6_rl_rllib\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load Fury Raid Sim Request JSON

In [2]:
f = open('data/fury-human-bis-p3.json')
settings = json.load(f)

# Set environment and settings

In [3]:
def reset():
    wowsims.new(json.dumps(settings).encode('utf-8'))


# Iterations are currently capped at 3000
iterations = settings['simOptions']['iterations']

duration = settings['encounter']['duration']
reset()

# Default sim agent
Default sim agent is the hardcoded agent from the sim

In [4]:
settings['simOptions']['interactive'] = False
damages = np.array([])

for i in tqdm(range(iterations)):
    reset()
    while not wowsims.step():
        pass
    totalDamage = wowsims.getDamageDone()
    damages = np.append(damages, totalDamage)

100%|██████████| 3000/3000 [00:08<00:00, 333.70it/s]


In [5]:
settings['simOptions']['interactive'] = False

reset()
while not wowsims.step():
    pass
    
    
cast_metrics = wowsims.getSpellMetrics()

for spell_id, metrics in cast_metrics.items():
    try:
        spell_name = spell_id_lists[spell_id]
    except KeyError as key:
        spell_name = key
    print(f"{spell_name}: {[metric['Casts'] for metric in metrics]}")

print(f'Average DPS: {damages.mean() / duration}')

Melee: [1, 1, 1, 2, 9, 77]
Death Wish: [2]
Deep Wounds: [133]
Whirlwind: [16]
Recklessness: [1]
Bloodthirst: [33]
Battle Stance: [9]
Berserker Stance: [9]
Bloodrage: [4]
Bloodlust: [1]
Whirlwind OH: [16]
Heroic Strike: [68]
Rend: [4]
Execute: [8]
Slam: [19]
Engi Gloves: [3]
Shattering Throw: [1]
Overpower: [5]
Average DPS: 11891.96688184432


# PPO Agent

## Load Trained PPO Model

In [6]:
config = PPOConfig()\
            .rollouts(num_rollout_workers=1)


register(id="FurySimEnv", entry_point="fury_sim_env:FurySimEnv")
env = make("FurySimEnv")
# env_creator = lambda config: FrameStack(env, num_stack=5)
env_creator = lambda config: FurySimEnv(...)
register_env("FurySimEnv", env_creator=env_creator)
# env = FrameStack(env, num_stack=5)

algorithm = config.build(env="FurySimEnv")

algorithm.from_checkpoint("models\\negative-failed-bloodsurge-reward\PPO\PPO_FurySimEnv_e5fd8_00000_0_2023-05-23_00-55-55\checkpoint_000078")

<>:14: DeprecationWarning: invalid escape sequence \P
2023-05-23 08:21:27,905	INFO worker.py:1616 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
2023-05-23 08:21:33,874	WARNING util.py:67 -- Install gputil for GPU system monitoring.
(RolloutWorker pid=20292) 2023-05-23 08:21:33,828	WARNING env.py:155 -- Your env doesn't have a .spec.max_episode_steps attribute. Your horizon will default to infinity, and your environment will not be reset.


ValueError: Given checkpoint (models
egative-failed-bloodsurge-reward\PPO\PPO_FurySimEnv_e5fd8_00000_0_2023-05-23_00-55-55\checkpoint_000078) not found! Must be a checkpoint directory (or a file for older checkpoint versions).

## Run PPO Model

In [ ]:
num_iteration = 3000
for i in tqdm(range(1, num_iteration+1)):
    observation, info = env.reset()
    terminated = False
    dps_results = np.array([])
    metrics_batch = []
    debug_logs = []
    debug_logs_batch = []
    state_out = []
    while not terminated:
        action = algorithm.compute_single_action(observation)
        observation, reward, terminated, truncated, info = env.step(action)
        debug_logs.append(info['debug log'])
    debug_logs_batch.append(debug_logs)
    dps_results = np.append(dps_results, info['dps'])
    metrics_batch.append(info['spell metrics'])

100%|██████████| 3000/3000 [24:58<00:00,  2.00it/s]


In [ ]:
median = np.argsort(dps_results)[len(dps_results)//2]
print(f'Average DPS: {dps_results[median]}')

cast_metrics = metrics_batch[median]
for spell_id, metrics in cast_metrics.items():
    # Only one target, so we can just take the first one
    try:
        spell_name = spell_id_lists[spell_id]
    except KeyError as key:
        spell_name = key
    print(f"{spell_name}: {[metric['Casts'] for metric in metrics]}")

Average DPS: 8817.386203732452
Melee: [1, 23, 39]
Death Wish: [1]
Deep Wounds: [70]
Whirlwind: [8]
Recklessness: [1]
Bloodthirst: [12]
Bloodrage: [2]
Bloodlust: [1]
Whirlwind OH: [8]
Heroic Strike: [16]
Execute: [3]
Slam: [16]
Engi Gloves: [2]


### Debug Log

In [18]:
actions_id = {
    0: "Bloodthirst", 
    1: "Whirlwind",
    2: "Slam",
    3: "HeroicStrike",
    4: "Execute",
    5: "DeathWish",
    6: "Recklessness",
    7: "ShatteringThrow",
    8: "Bloodrage",
    9: "EngiGlove",
    10: "Bloodlust",
    11: "Idle"
}

In [19]:
import pandas as pd

df = pd.DataFrame(debug_logs_batch[median], columns=['Timestamp', 'Action', 'Successful Cast', 'Damage Done', 'Total Damage Done', 'Rage'])
df['Action'] = df['Action'].map(actions_id)
df['DPS'] = df['Total Damage Done'].div(df['Timestamp'])
df

,Timestamp,Action,Successful Cast,Damage Done,Total Damage Done,Rage,DPS
0,0.000000,DeathWish,True,3034.868036,3034.868036,0.214065,inf
1,0.000000,DeathWish,False,0.000000,3034.868036,0.214065,inf
2,0.000000,Slam,False,0.000000,3034.868036,0.214065,inf
3,0.010000,Whirlwind,False,0.000000,3034.868036,0.214065,3.034868e+05
4,0.342143,HeroicStrike,True,7770.345572,10805.213608,0.982879,3.158101e+04
...,...,...,...,...,...,...,...
271,90.146282,Bloodthirst,False,0.000000,798545.121136,0.800000,8.858326e+03
272,90.146282,Bloodthirst,False,0.000000,798545.121136,0.800000,8.858326e+03
273,90.527540,Whirlwind,False,3009.422791,801554.543928,0.680000,8.854262e+03
274,91.146282,Execute,False,2117.421850,803671.965778,0.680000,8.817386e+03


In [11]:
df.to_csv("debug_logs/debug_log_1.csv")

# Spell Debug

In [ ]:
reset()
Spells.register()
Auras.register()
TargetAuras.register()

settings['simOptions']['interactive'] = True
damages = np.array([])
spell_metrics = []

for i in range(1):
    reset()
    while not wowsims.step():
        print(env.reset())
        if wowsims.needsInput():
            wowsims.trySpell(Spells.Bloodthirst)
    totalDamage = wowsims.getDamageDone()
    damages = np.append(damages, totalDamage)
    spell_metrics.append(wowsims.getSpellMetrics())

median = np.argsort(damages)[len(damages)//2]
print(f'Average DPS: {damages[median] / duration}')

In [7]:
settings['simOptions']['interactive'] = True

reset()
while not wowsims.step():
    pass
cast_metrics = spell_metrics[median]
for spell_id, metrics in cast_metrics.items():
    try:
        spell_name = spell_id_lists[spell_id]
    except KeyError as key:
        spell_name = key
    print(f"{spell_name}: {[metric['Casts'] for metric in metrics]}")

Melee: [1, 54]
Deep Wounds: [67]
Heroic Strike: [54]
